In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Delivery Route Optimization Analysis with Google Gemini\n",
    "\n",
    "This notebook runs the reinforcement learning simulation for delivery route optimization and uses Google's Gemini AI to interpret the results.\n",
    "\n",
    "### Prerequisites\n",
    "Ensure you have the following installed:\n",
    "`pip install google-generativeai osmnx networkx folium torch matplotlib numpy`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from IPython.display import Markdown, display\n",
    "\n",
    "# Import project modules\n",
    "from main import generate_random_locations, train_tabular_agent, train_dqn_agent, evaluate_agent\n",
    "from environment import DeliveryEnvironment\n",
    "from q_learning import QLearningAgent\n",
    "from sarsa import SarsaAgent\n",
    "from dqn import DQNAgent\n",
    "from main_with_google_ai import GoogleAIModelExplainer\n",
    "\n",
    "# --- SETUP API KEY ---\n",
    "# If you haven't set GOOGLE_API_KEY in your environment variables, uncomment and set it here:\n",
    "# os.environ[\"GOOGLE_API_KEY\"] = \"YOUR_ACTUAL_API_KEY_HERE\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# --- CONFIGURATION ---\n",
    "CITY = \"Middlesbrough\"\n",
    "NUM_PARCELS = 10  # Reduced for faster execution in notebook\n",
    "DISTANCE_METRIC = 'network' # 'network', 'manhattan', or 'driving'\n",
    "NUM_EPISODES = 1000"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# --- INITIALIZE ENVIRONMENT ---\n",
    "print(f\"Generating locations in {CITY}...\")\n",
    "locations = generate_random_locations(CITY, NUM_PARCELS + 1)\n",
    "\n",
    "if locations is not None:\n",
    "    print(f\"Initializing Environment with {DISTANCE_METRIC} metric...\")\n",
    "    env = DeliveryEnvironment(locations=locations, city_name=CITY, distance_metric=DISTANCE_METRIC)\n",
    "    print(f\"Environment ready. {env.num_locations} locations.\")\n",
    "else:\n",
    "    print(\"Failed to generate locations.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# --- INITIALIZE AGENTS ---\n",
    "agents = {\n",
    "    \"Q-Learning\": QLearningAgent(action_space=list(range(env.num_locations))),\n",
    "    \"SARSA\": SarsaAgent(action_space=list(range(env.num_locations))),\n",
    "    \"DQN\": DQNAgent(state_size=env.get_state_size(), action_size=env.num_locations)\n",
    "}"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# --- TRAINING LOOP ---\n",
    "reward_histories = {}\n",
    "final_results = {}\n",
    "best_route_info = {\"agent\": None, \"route\": [], \"distance\": float('inf')}\n",
    "\n",
    "for name, agent in agents.items():\n",
    "    print(f\"Training {name} for {NUM_EPISODES} episodes...\")\n",
    "    \n",
    "    if isinstance(agent, DQNAgent):\n",
    "        reward_histories[name] = train_dqn_agent(agent, env, NUM_EPISODES)\n",
    "    else:\n",
    "        reward_histories[name] = train_tabular_agent(agent, env, NUM_EPISODES)\n",
    "    \n",
    "    # Evaluate\n",
    "    route, distance = evaluate_agent(agent, env)\n",
    "    \n",
    "    # Calculate metrics for AI analysis\n",
    "    avg_reward = np.mean(reward_histories[name][-100:])\n",
    "    final_results[name] = {\n",
    "        \"total_distance_km\": distance,\n",
    "        \"avg_final_reward\": avg_reward,\n",
    "        \"convergence_speed\": \"Fast\" if avg_reward > -1000 else \"Slow\" # Simple heuristic\n",
    "    }\n",
    "    \n",
    "    print(f\"  > Final Distance: {distance:.2f} km\")\n",
    "    \n",
    "    if distance < best_route_info[\"distance\"]:\n",
    "        best_route_info = {\"agent\": name, \"route\": route, \"distance\": distance}\n",
    "\n",
    "print(\"\\nTraining Complete.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# --- VISUALIZATION ---\n",
    "plt.figure(figsize=(12, 6))\n",
    "for name, rewards in reward_histories.items():\n",
    "    # Smooth curves\n",
    "    window = 50\n",
    "    if len(rewards) >= window:\n",
    "        avg_rewards = np.convolve(rewards, np.ones(window)/window, mode='valid')\n",
    "        plt.plot(avg_rewards, label=name)\n",
    "    else:\n",
    "        plt.plot(rewards, label=name)\n",
    "\n",
    "plt.title(\"Agent Learning Curves\")\n",
    "plt.xlabel(\"Episode\")\n",
    "plt.ylabel(\"Reward (Moving Avg)\")\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# --- GOOGLE GEMINI ANALYSIS ---\n",
    "print(\"Initializing Google AI Explainer...\")\n",
    "explainer = GoogleAIModelExplainer()\n",
    "\n",
    "env_config = {\n",
    "    \"city\": CITY,\n",
    "    \"num_parcels\": NUM_PARCELS,\n",
    "    \"distance_metric\": DISTANCE_METRIC,\n",
    "    \"episodes\": NUM_EPISODES\n",
    "}\n",
    "\n",
    "if explainer.available:\n",
    "    print(\"\\n--- AI Performance Analysis ---\")\n",
    "    analysis = explainer.analyze_performance(final_results, env_config)\n",
    "    display(Markdown(analysis))\n",
    "    \n",
    "    print(\"\\n--- Full Training Report ---\")\n",
    "    report = explainer.generate_training_report(reward_histories, final_results, env_config)\n",
    "    display(Markdown(report))\n",
    "else:\n",
    "    print(\"Google AI not available. Please check your API key.\")\n",
    "    print(\"Results Summary:\", final_results)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.13"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}